In [1]:
import logging


def get_logger(name: str):
    logger = logging.getLogger(name)
    if not logger.handlers:  # Prevents duplicate handlers
        logger.setLevel(logging.DEBUG)  # Set the logger level to DEBUG
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)  # Set the handler level to INFO
        formatter = logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        )  # Define the log message format
        ch.setFormatter(formatter)
        logger.addHandler(ch)
    return logger


In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")

DAPPIER_API_KEY = os.getenv("DAPPIER_API_KEY")

AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY")


In [12]:
from camel.toolkits import DappierToolkit, FunctionTool

from utils.logger import get_logger

logger = get_logger(__name__)


class WeatherToolWrapper:
    def __init__(self):
        self.toolkit = DappierToolkit()
        logger.info("Weather Tool initialized")

    def get_tool(self):
        return FunctionTool(self.search_weather)

    def search_weather(self, query: str, city: str):
        full_query = f"{query} in {city}"
        logger.debug("WeatherTool query: %s", full_query)
        try:
            response = self.toolkit.search_real_time_data(query=query)
            logger.debug("WeatherTool response: %s", response)
            return response
        except Exception as e:
            logger.error("WeatherTool error: %s", str(e))
            return "Error retrieving weather data."


In [14]:
from camel.toolkits import DappierToolkit, FunctionTool
from utils.logger import get_logger

logger = get_logger(__name__)

class TrafficToolWrapper:
    def __init__(self):
        self.toolkit = DappierToolkit()
        logger.info("Traffic Tool initialized")
    
    def get_tool(self):
        return FunctionTool(self.search_traffic)

    def search_traffic(self, query: str, city: str):
        full_query = f"{query} in {city}"
        logger.debug("TrafficTool query: %s", full_query)
        try:
            response = self.toolkit.search_real_time_data(query=full_query)
            logger.debug("TrafficTool response: %s", response)
            return response
        except Exception as e:
            logger.error("TrafficTool error: %s", str(e))
            return "Error retrieving traffic data."


In [15]:
# Test TrafficAgent
traffic_agent = TrafficAgent(city)
print(traffic_agent.get_traffic_status())


🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
2025-03-06 04:21:31,171 - __main__ - INFO - Traffic Tool initialized


2025-03-06 04:21:31,171 - __main__ - INFO - Traffic Tool initialized
2025-03-06 04:21:31,178 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.


/home/parthshr370/anaconda3/envs/agents/lib/python3.11/site-packages/camel/toolkits/function_tool.py:427: UserWarning: Function description is missing for 
                          search_traffic. This may 
                          affect the quality of tool calling.
  warnings.warn(f"""Function description is missing for
/home/parthshr370/anaconda3/envs/agents/lib/python3.11/site-packages/camel/toolkits/function_tool.py:448: UserWarning: Parameter description is missing for 
                            {'type': 'string'}. This may affect the quality of tool 
                            calling.
  warnings.warn(f"""Parameter description is missing for
2025-03-06 04:21:34,508 - __main__ - INFO - TrafficAgent response: msgs=[BaseMessage(role_name='Traffic Agent', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)] terminated=False info={'id': 'gen-1741215092-QieXsJygzk5pB1mslfk1

2025-03-06 04:21:34,508 - __main__ - INFO - TrafficAgent response: msgs=[BaseMessage(role_name='Traffic Agent', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)] terminated=False info={'id': 'gen-1741215092-QieXsJygzk5pB1mslfk1', 'usage': {'completion_tokens': 0, 'prompt_tokens': 299, 'total_tokens': 299, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 285, 'tool_calls': [], 'external_tool_request': None}
{'traffic_status': ChatAgentResponse(msgs=[BaseMessage(role_name='Traffic Agent', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)], terminated=False, info={'id': 'gen-1741215092-QieXsJygzk5pB1mslfk1', 'usage': {'completion_tokens': 0, 'prompt_tokens': 299, 'total_tokens': 299, 'completion_tokens_detai

In [5]:
'''tool = TrafficToolWrapper()
result = tool.search_traffic("traffic", "New York")
print(result)
'''

'tool = TrafficToolWrapper()\nresult = tool.search_traffic("traffic", "New York")\nprint(result)\n'

In [6]:
def aggregate_city_data(citizen_data, traffic_data, env_data):
    # Combine data from all agents into a city dashboard summary.
    dashboard = "=== Smart City Dashboard ===\n\n"
    dashboard += (
        "Citizen Feedback:\n" + citizen_data.get("citizen_feedback", "No data") + "\n\n"
    )

    dashboard += (
        "Traffic Status:\n" + traffic_data.get("traffic_status", "No data") + "\n\n"
    )
    dashboard += (
        "Environment Update:\n" + env_data.get("environment_update", "No data") + "\n"
    )
    return dashboard


In [7]:
from camel.agents.chat_agent import ChatAgent
from camel.messages.base import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from config import OPEN_ROUTER_API_KEY
from tools.traffic_tool import TrafficToolWrapper
from utils.logger import get_logger

logger = get_logger(__name__)

class TrafficAgent:
    def __init__(self, city):
        self.city = city
        self.model = ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
            api_key=OPEN_ROUTER_API_KEY,
            model_type="google/gemini-2.0-flash-001",
            url="https://openrouter.ai/api/v1",
            model_config_dict={"temperature": 0.4, "max_tokens": 4096},
        )

        sys_msg = BaseMessage.make_assistant_message(
            role_name="Traffic Agent",
            content=f"You analyze current traffic conditions and congestion in the city {self.city}.",
        )

        self.traffic_tool = TrafficToolWrapper()
        self.agent = ChatAgent(
            system_message=sys_msg,
            model=self.model,
            tools=[self.traffic_tool.get_tool()],
        )

    def get_traffic_status(self):
        prompt = f"What is the current traffic congestion status in {self.city}?"
        response = self.agent.step(prompt)
        logger.info("TrafficAgent response: %s", response)
        return {"traffic_status": response}


In [16]:
from camel.agents.chat_agent import ChatAgent
from camel.configs import ChatGPTConfig
from camel.messages.base import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

from config import OPEN_ROUTER_API_KEY
from tools.weather_tool import WeatherToolWrapper
from utils.logger import get_logger

logger = get_logger(__name__)


class EnvironmentAgent:
    def __init__(self, city):
        self.city = city
        self.model = ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
            api_key=OPEN_ROUTER_API_KEY,
            model_type="google/gemini-2.0-flash-001",
            url="https://openrouter.ai/api/v1",
            model_config_dict={"temperature": 0.4, "max_tokens": 4096},
        )

        sys_msg = BaseMessage.make_assistant_message(
            role_name="Environment Agent",
            content=f"You provide real-time weather and environmental updates for {self.city}.",
        )

        self.weather_tool = WeatherToolWrapper()
        self.agent = ChatAgent(
            system_message=sys_msg,
            model=self.model,
            tools=[self.weather_tool.get_tool()],
        )
        logger.info("Environment Agent initialized for city: %s", self.city)

    def get_environment_update(self):
        prompt = f"Provide the latest weather update and any notable environmental alerts for {self.city}."
        response = self.agent.step(prompt)
        logger.info("Environment update: %s", response)
        return {"environment_update": response}


In [17]:
class CitizenAgent:
    def __init__(self, city):
        self.city = city
        self.model = ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
            api_key=OPEN_ROUTER_API_KEY,
            model_type="google/gemini-2.0-flash-001",
            url="https://openrouter.ai/api/v1",
            model_config_dict={"temperature": 0.4, "max_tokens": 4096},
        )

        sys_msg = BaseMessage.make_assistant_message(
            role_name="Citizen Agent",
            content=f"You simulate citizen feedback on urban issues (e.g., public events, safety, local services) in {self.city}.",
        )

        self.agent = ChatAgent(system_message=sys_msg, model=self.model)
        logger.info("Citizen Agent initialized for city: %s", self.city)

    def get_city_feedback(self):
        prompt = f"Provide recent citizen feedback on city services and public safety in {self.city}."
        response = self.agent.step(prompt)
        logger.info("City feedback: %s", response)
        return {"citizen_feedback": response}


In [10]:
city = "Ney York"

In [18]:
environment_agent = EnvironmentAgent(city)
print(environment_agent.get_environment_update())

🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
2025-03-06 04:22:05,605 - tools.weather_tool - INFO - Weather Tool initialized


2025-03-06 04:22:05,605 - tools.weather_tool - INFO - Weather Tool initialized
2025-03-06 04:22:05,608 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.


/home/parthshr370/anaconda3/envs/agents/lib/python3.11/site-packages/camel/toolkits/function_tool.py:427: UserWarning: Function description is missing for 
                          search_weather. This may 
                          affect the quality of tool calling.
  warnings.warn(f"""Function description is missing for
2025-03-06 04:22:05,610 - __main__ - INFO - Environment Agent initialized for city: Ney York


2025-03-06 04:22:05,610 - __main__ - INFO - Environment Agent initialized for city: Ney York


2025-03-06 04:22:11,638 - __main__ - INFO - Environment update: msgs=[BaseMessage(role_name='Environment Agent', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)] terminated=False info={'id': 'gen-1741215128-U7yOpfJKfWXVYYKDk8sr', 'usage': {'completion_tokens': 0, 'prompt_tokens': 302, 'total_tokens': 302, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 285, 'tool_calls': [], 'external_tool_request': None}


2025-03-06 04:22:11,638 - __main__ - INFO - Environment update: msgs=[BaseMessage(role_name='Environment Agent', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)] terminated=False info={'id': 'gen-1741215128-U7yOpfJKfWXVYYKDk8sr', 'usage': {'completion_tokens': 0, 'prompt_tokens': 302, 'total_tokens': 302, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 285, 'tool_calls': [], 'external_tool_request': None}
{'environment_update': ChatAgentResponse(msgs=[BaseMessage(role_name='Environment Agent', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)], terminated=False, info={'id': 'gen-1741215128-U7yOpfJKfWXVYYKDk8sr', 'usage': {'completion_tokens': 0, 'prompt_tokens': 302, 'total_tokens': 302, 'completion_tok

In [36]:
print(dir(citizen_agent.agent))


['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_agentops_agent_id', '_agentops_agent_name', '_extract_tool_call', '_generate_tool_prompt', '_handle_step', '_initialize_tools', '_inject_tool_prompt', '_is_standard_response', '_log_final_output', '_parse_tool_response', '_safe_json_loads', '_safe_model_dump', '_step_get_info', '_step_model_response', '_step_token_exceed', '_step_tool_call', '_step_tool_call_and_update', '_step_tool_call_and_update_async', '_structure_output_with_function', '_system_message', 'add_model_scheduling_strategy', 'add_tool', 'agentops_agent_id', 'agentops_agent_name', 'all_tools', 'ex

In [33]:
import agentops
from utils.logger import get_logger
from agents.citizen_agent import CitizenAgent
from agents.traffic_agent import TrafficAgent
from agents.environment_agent import EnvironmentAgent
from tools.data_aggregator import aggregate_city_data

logger = get_logger(__name__)

def main():
    agentops.init(default_tags=["City Emulator"])
    logger.info("Starting City Emulator")
    
    city = input("Enter the city for which you want to retrieve data (default: NYC): ") or "NYC"
    
    logger.info("Instantiating agents...")
    citizen_agent = CitizenAgent(city)
    traffic_agent = TrafficAgent(city)
    environment_agent = EnvironmentAgent(city)
    
    logger.info("Collecting citizen feedback...")
    citizen_data = citizen_agent.get_city_feedback()

    logger.info("Collecting traffic data...")
    traffic_data = traffic_agent.get_traffic_status()

    logger.info("Collecting environmental updates...")
    env_data = environment_agent.get_environment_update()

    logger.info("Aggregating city data...")
    dashboard = aggregate_city_data(citizen_data, traffic_data, env_data)
    print("\n--- City Dashboard ---\n")
    print(dashboard)

    agentops.end_session("Success")
    logger.info("AgentOps session ended successfully.")

if __name__ == "__main__":
    main()


🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.
2025-03-06 03:57:19,704 - __main__ - INFO - Starting City Emulator


2025-03-06 03:57:19,704 - __main__ - INFO - Starting City Emulator


2025-03-06 03:57:21,919 - __main__ - INFO - Instantiating agents...


2025-03-06 03:57:21,919 - __main__ - INFO - Instantiating agents...


2025-03-06 03:57:22,688 - agents.citizen_agent - INFO - Citizen Agent initialized for city: nyc


2025-03-06 03:57:22,688 - agents.citizen_agent - INFO - Citizen Agent initialized for city: nyc


TypeError: TrafficAgent.__init__() takes 1 positional argument but 2 were given

In [ ]:
from camel.types import ModelType
print(list(ModelType))


[<ModelType.GPT_4O_MINI: 'gpt-4o-mini'>, <ModelType.GPT_3_5_TURBO: 'gpt-3.5-turbo'>, <ModelType.GPT_4: 'gpt-4'>, <ModelType.GPT_4_TURBO: 'gpt-4-turbo'>, <ModelType.GPT_4O: 'gpt-4o'>, <ModelType.O1: 'o1'>, <ModelType.O1_PREVIEW: 'o1-preview'>, <ModelType.O1_MINI: 'o1-mini'>, <ModelType.O3_MINI: 'o3-mini'>, <ModelType.GLM_4: 'glm-4'>, <ModelType.GLM_4V: 'glm-4v'>, <ModelType.GLM_4V_FLASH: 'glm-4v-flash'>, <ModelType.GLM_4V_PLUS_0111: 'glm-4v-plus-0111'>, <ModelType.GLM_4_PLUS: 'glm-4-plus'>, <ModelType.GLM_4_AIR: 'glm-4-air'>, <ModelType.GLM_4_AIR_0111: 'glm-4-air-0111'>, <ModelType.GLM_4_AIRX: 'glm-4-airx'>, <ModelType.GLM_4_LONG: 'glm-4-long'>, <ModelType.GLM_4_FLASHX: 'glm-4-flashx'>, <ModelType.GLM_4_FLASH: 'glm-4-flash'>, <ModelType.GLM_ZERO_PREVIEW: 'glm-zero-preview'>, <ModelType.GLM_3_TURBO: 'glm-3-turbo'>, <ModelType.GROQ_LLAMA_3_1_8B: 'llama-3.1-8b-instant'>, <ModelType.GROQ_LLAMA_3_3_70B: 'llama-3.3-70b-versatile'>, <ModelType.GROQ_LLAMA_3_3_70B_PREVIEW: 'llama-3.3-70b-specdec